## Importing modules

In [1]:
!pip install scikit-learn xgboost catboost lightgbm category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 59.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 6.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import pycaret
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, f1_score, precision_score, recall_score, roc_curve, roc_auc_score, make_scorer
from sklearn.metrics import matthews_corrcoef
from category_encoders import WOEEncoder, BinaryEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer, PolynomialFeatures
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split, StratifiedKFold, LeaveOneOut, cross_validate
import pickle
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

## Loading the dataset

In [4]:
ML_data = pd.read_csv("/Users/kassywang/Documents/Summer Project/R/without outliers/Clean_complete_data_by_dipping_catergory.csv")
ML_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   study_id                    699 non-null    object 
 1   ageyrs                      699 non-null    int64  
 2   sex                         699 non-null    int64  
 3   finalhiv                    699 non-null    int64  
 4   artyrs                      699 non-null    float64
 5   fruit                       699 non-null    int64  
 6   veg                         699 non-null    int64  
 7   salt                        699 non-null    int64  
 8   beverage                    699 non-null    int64  
 9   snacks                      699 non-null    int64  
 10  friedfood                   699 non-null    int64  
 11  creatinine                  699 non-null    float64
 12  triglycerides               699 non-null    float64
 13  low-density lipoprotein     699 non

In [5]:
#### Checking the shape of the dataset
print("Shape of the dataset:", ML_data.shape)

Shape of the dataset: (699, 26)


In [6]:
## Checking the unique catergories of dipping_ctergory
ML_data['calculated_classification'].unique()

array([0, 1, 3, 2])

In [7]:
## Obtaining there count
ML_data['calculated_classification'].value_counts()

calculated_classification
1    367
0    257
3     54
2     21
Name: count, dtype: int64

## Converting the data to the correct data type

In [9]:
# Columns to exclude from categorical conversion
exclude_columns = ["ageyrs",'artyrs', 'bmi','hivyrs','waist_hip_ratio', 'creatinine', 'triglycerides', 'low-density lipoprotein',
       'total cholesterol', 'high-density lipoprotein', 'eGFR_2021']

# Select all columns except the ones in exclude_columns
columns_to_convert = ML_data.columns.difference(exclude_columns)

# Convert selected columns to categorical
ML_data[columns_to_convert] = ML_data[columns_to_convert].astype('category')

In [10]:
ML_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   study_id                    699 non-null    category
 1   ageyrs                      699 non-null    int64   
 2   sex                         699 non-null    category
 3   finalhiv                    699 non-null    category
 4   artyrs                      699 non-null    float64 
 5   fruit                       699 non-null    category
 6   veg                         699 non-null    category
 7   salt                        699 non-null    category
 8   beverage                    699 non-null    category
 9   snacks                      699 non-null    category
 10  friedfood                   699 non-null    category
 11  creatinine                  699 non-null    float64 
 12  triglycerides               699 non-null    float64 
 13  low-density lipoprot

In [11]:
### Delete the study_id and unnamed columms
ML_data = ML_data.drop(columns=['study_id', 'calculated_dipping_status'])
ML_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   ageyrs                      699 non-null    int64   
 1   sex                         699 non-null    category
 2   finalhiv                    699 non-null    category
 3   artyrs                      699 non-null    float64 
 4   fruit                       699 non-null    category
 5   veg                         699 non-null    category
 6   salt                        699 non-null    category
 7   beverage                    699 non-null    category
 8   snacks                      699 non-null    category
 9   friedfood                   699 non-null    category
 10  creatinine                  699 non-null    float64 
 11  triglycerides               699 non-null    float64 
 12  low-density lipoprotein     699 non-null    float64 
 13  total cholesterol   

In [12]:
### Create a new dataset called clean_data as a copy of the ML_data
clean_data = ML_data.copy()

In [13]:
#### Checking for missing values in the dataset
missing_values = clean_data.isnull().sum()
print("Missing values in each column:\n", missing_values[missing_values > 0])

Missing values in each column:
 Series([], dtype: int64)


In [14]:
clean_data.describe().T

,count,mean,std,min,25%,50%,75%,max
ageyrs,699.0,36.570815,12.037980,17.00,25.00,38.00,46.00,76.00
artyrs,699.0,1.679542,5.270372,0.00,0.00,0.00,0.00,98.00
creatinine,699.0,0.826609,0.205390,0.30,0.70,0.80,0.90,3.30
triglycerides,699.0,1.338770,0.770721,0.10,0.80,1.10,1.60,8.60
low-density lipoprotein,699.0,2.446209,0.896936,0.20,1.80,2.40,3.00,6.80
total cholesterol,699.0,4.253505,1.191336,0.10,3.40,4.20,4.90,9.70
high-density lipoprotein,699.0,1.180544,0.392179,0.30,0.90,1.10,1.40,3.90
eGFR_2021,699.0,105.462518,19.245385,21.50,92.65,108.50,120.10,149.90
bmi,699.0,23.938913,13.211905,10.00,20.60,22.40,25.75,354.80
waist_hip_ratio,699.0,0.889256,0.631916,0.47,0.80,0.85,0.89,11.16


In [15]:
## Check the clean data for the row that has value as 98.00 for artyrs and its finalhiv
clean_data[clean_data['artyrs'] == 98.00]

,ageyrs,sex,finalhiv,artyrs,fruit,veg,salt,beverage,snacks,friedfood,...,high-density lipoprotein,eGFR_2021,vigorous physical activity,moderate physical activity,walking,calculated_classification,smoking,bmi,waist_hip_ratio,alcohol_consumption
404,26,0,0,98.0,1,0,1,1,1,1,...,0.8,149.9,0,1,0,0,0,27.6,9.7,0


In [16]:
clean_data.loc[clean_data['artyrs'] == 98.00, 'artyrs'] = 0
clean_data.describe().T

,count,mean,std,min,25%,50%,75%,max
ageyrs,699.0,36.570815,12.037980,17.00,25.00,38.00,46.00,76.00
artyrs,699.0,1.539342,3.803872,0.00,0.00,0.00,0.00,25.00
creatinine,699.0,0.826609,0.205390,0.30,0.70,0.80,0.90,3.30
triglycerides,699.0,1.338770,0.770721,0.10,0.80,1.10,1.60,8.60
low-density lipoprotein,699.0,2.446209,0.896936,0.20,1.80,2.40,3.00,6.80
total cholesterol,699.0,4.253505,1.191336,0.10,3.40,4.20,4.90,9.70
high-density lipoprotein,699.0,1.180544,0.392179,0.30,0.90,1.10,1.40,3.90
eGFR_2021,699.0,105.462518,19.245385,21.50,92.65,108.50,120.10,149.90
bmi,699.0,23.938913,13.211905,10.00,20.60,22.40,25.75,354.80
waist_hip_ratio,699.0,0.889256,0.631916,0.47,0.80,0.85,0.89,11.16


#### **Define the features and target variables**

In [17]:
X = clean_data.drop('calculated_classification', axis=True)
Y = clean_data['calculated_classification']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

print("Shape of training set:", X_train.shape)
print("Shape of testing set:", X_test.shape)

train_dipping_counts = y_train.value_counts()
print("Dipping category distribution in training set:\n", train_dipping_counts)

test_dipping_counts = y_test.value_counts()
print("Dipping category distribution in testing set:\n", test_dipping_counts)

Shape of training set: (559, 23)
Shape of testing set: (140, 23)
Dipping category distribution in training set:
 calculated_classification
1    293
0    206
3     43
2     17
Name: count, dtype: int64
Dipping category distribution in testing set:
 calculated_classification
1    74
0    51
3    11
2     4
Name: count, dtype: int64


#### **Model training on an imbalanced dataset**

### List of the ML algorithms for model training

In [19]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

model_knn=KNeighborsClassifier()
model_nb=GaussianNB()
model_mlp = MLPClassifier(random_state=1)
model_knn = KNeighborsClassifier()
model_rf = RandomForestClassifier()
model_logr =LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=0)
model_lgbm = LGBMClassifier(random_state=0, n_jobs = -1)
model_xgb = XGBClassifier(random_state=0)
model_gbc = GradientBoostingClassifier(random_state=0)
model_cat = CatBoostClassifier(verbose=False)
model_dt = DecisionTreeClassifier()
model_et = ExtraTreesClassifier()
model_svm = SVC(probability=True)
model_ada = AdaBoostClassifier(random_state=0)

models = []
models.append(('Logistic regression', model_logr))
models.append(('Suport Vector Machine', model_svm))
models.append(('Decision trees', model_dt))
models.append(('Random Forest', model_rf))
models.append(('Ada Boost', model_ada))
models.append(('Gradient Boosting', model_gbc))
models.append(('ExtraTrees Classifier', model_et))
models.append(('LightGBM', model_lgbm))
models.append(('CatBoost', model_cat))
models.append(('XGBoost', model_xgb))
models.append(('MLPClassifier', model_mlp))
models.append(('KNeighborsClassifier', model_knn))
models.append(('Gaussian NB', model_nb))

In [20]:
clean_data.columns

Index(['ageyrs', 'sex', 'finalhiv', 'artyrs', 'fruit', 'veg', 'salt',
       'beverage', 'snacks', 'friedfood', 'creatinine', 'triglycerides',
       'low-density lipoprotein', 'total cholesterol',
       'high-density lipoprotein', 'eGFR_2021', 'vigorous physical activity',
       'moderate physical activity', 'walking', 'calculated_classification',
       'smoking', 'bmi', 'waist_hip_ratio', 'alcohol_consumption'],
      dtype='object')

In [21]:
### Define the categorical and numerical columns
cat_var = ['sex', 'finalhiv',
            'fruit', 'veg', 'salt', 'beverage',
            'snacks', 'friedfood','smoking',
            'alcohol_consumption','vigorous physical activity',
       'moderate physical activity', 'walking']

con_var = ['bmi', 'ageyrs', 'artyrs','creatinine', 'triglycerides',
       'low-density lipoprotein', 'total cholesterol', 'high-density lipoprotein', 'eGFR_2021', 'waist_hip_ratio']

#### **Creating the pipeline and Feature Scaling**

In [22]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectFromModel

scaler = StandardScaler()
onehot = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
cv = StratifiedKFold(n_splits=3, random_state=1, shuffle=True)
#feature = SelectFromModel(model_rf, threshold=0.001)
ct = ColumnTransformer([('onehot', onehot, cat_var), # only multi-category vars
 ('scaler', scaler, con_var)], remainder='passthrough', n_jobs=-1)


#### **Cross validation**

In [23]:
# Setting the StratifiedKFold parameter
skfold = StratifiedKFold(n_splits = 3, random_state = 42, shuffle = True)

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import (
    make_scorer,
    matthews_corrcoef,
    confusion_matrix,
    recall_score,
    precision_score,
    f1_score,
    roc_auc_score
)
from sklearn.pipeline import Pipeline


# Function to compute bootstrap confidence interval
def bootstrap_confidence_interval(data, confidence=0.95, n_bootstrap=1000):
    """Compute bootstrap confidence intervals for a metric."""
    means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data, size=len(data), replace=True)
        means.append(np.mean(sample))
    lower = np.percentile(means, (1 - confidence) / 2 * 100)
    upper = np.percentile(means, (1 + confidence) / 2 * 100)
    return np.mean(means), lower, upper


# Specificity scorer for multiclass (macro-average)
def specificity_score(y_true, y_pred):
    """Macro-average specificity for multiclass classification."""
    cm = confusion_matrix(y_true, y_pred)
    specificities = []
    for i in range(len(cm)):
        tn = np.sum(np.delete(np.delete(cm, i, axis=0), i, axis=1))
        fp = np.sum(np.delete(cm[i, :], i))
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        specificities.append(specificity)
    return np.mean(specificities)


specificity_scorer = make_scorer(specificity_score)


# ROC AUC scorer for multiclass using callable

def roc_auc_scorer_callable(estimator, X, y):
    """Custom ROC AUC scorer for multiclass problems."""
    if hasattr(estimator, "predict_proba"):
        y_score = estimator.predict_proba(X)
    elif hasattr(estimator, "decision_function"):
        y_score = estimator.decision_function(X)
    else:
        return np.nan
    return roc_auc_score(y, y_score, multi_class='ovr', average='macro')



def cross_validate_metrics(models, X, y, cv=5, confidence=0.95, n_bootstrap=1000):
    """
    Perform CV for multiple models with multiple metrics.
    Returns:
        summary_df -> Table with mean (lower, upper CI)
        fold_df -> Raw scores for boxplots
    """
    skfold = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    scorers = {
        "accuracy": "accuracy",
        "precision": make_scorer(precision_score, average='macro', zero_division=0),
        "recall": make_scorer(recall_score, average='macro', zero_division=0),
        "f1": make_scorer(f1_score, average='macro', zero_division=0),
        "mcc": make_scorer(matthews_corrcoef),
        "specificity": specificity_scorer,
        "roc_auc": roc_auc_scorer_callable
    }

    summary_results = []
    all_folds = []  # store raw scores for plotting

    for name, classifier in models:
        print(f"\nEvaluating model: {name}")
        pipe = Pipeline([('ct', ct), (name, classifier)])  # assumes ct is predefined
        model_results = {metric: [] for metric in scorers}

        for score_name, scorer in scorers.items():
            cv_scores = cross_val_score(pipe, X, y, cv=skfold, scoring=scorer)
            model_results[score_name] = cv_scores

            # store each fold score for plotting
            for s in cv_scores:
                all_folds.append({
                    "model": name,
                    "metric": score_name,
                    "score": s
                })

            print(f"{score_name} : {np.nanmean(cv_scores):.4f}")

        # Create summary with mean, CI
        summary = {'model': name}
        for metric, scores in model_results.items():
            scores = np.array(scores)
            valid_scores = scores[~np.isnan(scores)]
            if len(valid_scores) > 0:
                mean, lower_ci, upper_ci = bootstrap_confidence_interval(
                    valid_scores, confidence, n_bootstrap
                )
                summary[f"{metric}"] = f"{mean:.3f} ({lower_ci:.3f}, {upper_ci:.3f})"
            else:
                summary[f"{metric}"] = "nan"
        summary_results.append(summary)

    return pd.DataFrame(summary_results), pd.DataFrame(all_folds)



In [25]:
# Run CV
summary_df, fold_scores_df = cross_validate_metrics(models, X, Y)

# View formatted table
print(summary_df.head(14))



Evaluating model: Logistic regression


/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/m

accuracy : 0.5151


/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/m

precision : 0.2440


...

mcc : 0.0336


/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization has

specificity : 0.7600


/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization has

roc_auc : 0.5027

Evaluating model: KNeighborsClassifier
accuracy : 0.4706
precision : 0.2297
recall : 0.2541
f1 : 0.2409
mcc : 0.0212
specificity : 0.7558
roc_auc : 0.4895

Evaluating model: Gaussian NB
accuracy : 0.0644
precision : 0.2592
recall : 0.2775
f1 : 0.0660
mcc : 0.0141
specificity : 0.7494
roc_auc : 0.5433
                    model              accuracy             precision  \
0     Logistic regression  0.516 (0.484, 0.546)  0.244 (0.225, 0.263)   
1   Suport Vector Machine  0.532 (0.504, 0.552)  0.257 (0.226, 0.283)   
2          Decision trees  0.463 (0.442, 0.484)  0.282 (0.255, 0.310)   
3           Random Forest  0.508 (0.490, 0.529)  0.250 (0.235, 0.265)   
4               Ada Boost  0.435 (0.383, 0.497)  0.245 (0.215, 0.273)   
5       Gradient Boosting  0.518 (0.501, 0.533)  0.266 (0.257, 0.284)   
6   ExtraTrees Classifier  0.475 (0.451, 0.499)  0.253 (0.214, 0.304)   
7                LightGBM  0.481 (0.452, 0.504)  0.236 (0.223, 0.249)   
8                CatBoo

In [26]:
### Save the output file
summary_df.to_csv('/Users/kassywang/Documents/Summer Project/R/without outliers/cross_validation_results.csv', index=False)

In [27]:
# Add fold number per model and metric (important to align folds correctly)
fold_scores_df['fold'] = fold_scores_df.groupby(['model', 'metric']).cumcount()

# Pivot so each metric becomes a column
pivot_df = fold_scores_df.pivot_table(
    index=['model', 'fold'],
    columns='metric',
    values='score'
).reset_index()

print(pivot_df.head(n =5))


metric      model  fold  accuracy        f1       mcc  precision    recall  \
0       Ada Boost     0  0.428571  0.235696  0.043824   0.244524  0.228991   
1       Ada Boost     1  0.350000  0.185532 -0.122675   0.186037  0.185788   
2       Ada Boost     2  0.414286  0.265521  0.045927   0.272196  0.307926   
3       Ada Boost     3  0.450000  0.228940 -0.002303   0.230384  0.235692   
4       Ada Boost     4  0.539568  0.289807  0.176976   0.289104  0.293782   

metric   roc_auc  specificity  
0       0.473132     0.765114  
1       0.468152     0.718011  
2       0.549590     0.759932  
3       0.460483     0.746546  
4       0.526467     0.796911  


In [28]:
# Assuming pivot_df is already created as shown before
selected_metrics = ['model', 'mcc', 'roc_auc']
subset_df = pivot_df[selected_metrics]
print(subset_df.head())


metric      model       mcc   roc_auc
0       Ada Boost  0.043824  0.473132
1       Ada Boost -0.122675  0.468152
2       Ada Boost  0.045927  0.549590
3       Ada Boost -0.002303  0.460483
4       Ada Boost  0.176976  0.526467


In [29]:
melted_df = pd.melt(
    subset_df,
    id_vars=['model'],      # columns to keep as identifiers
    value_vars=['mcc', 'roc_auc'],  # columns to unpivot
    var_name='metric',
    value_name='score'
)

melted_df.head(2)


,model,metric,score
0,Ada Boost,mcc,0.043824
1,Ada Boost,mcc,-0.122675


#### **Handling for class imbalance**

In [31]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [32]:
# define the catergorical columns
cat_features = ['sex', 'finalhiv',
            'fruit', 'veg', 'salt', 'beverage',
            'snacks', 'friedfood','smoking',
            'alcohol_consumption','vigorous physical activity',
       'moderate physical activity', 'walking']

#define the numerical columns
num_cols = ['bmi', 'ageyrs', 'artyrs','creatinine', 'triglycerides',
       'low-density lipoprotein', 'total cholesterol', 'high-density lipoprotein', 'eGFR_2021', 'waist_hip_ratio']

In [33]:
sampling_strategy = {0: 379, 1: 379, 2: 379, 3: 379}  # All classes to 310 samples

In [34]:
from imblearn.over_sampling import SMOTENC

# Initialize SMOTENC
smote_nc = SMOTENC(categorical_features=cat_features,
                   sampling_strategy=sampling_strategy,
                   random_state=42)
X_resampled, y_resampled = smote_nc.fit_resample(X_train, y_train)

In [35]:
X_resampled.shape, y_resampled.shape

((1516, 23), (1516,))

In [36]:
print("After SMOTE - Training set distribution:")
print(pd.Series(y_resampled).value_counts())

After SMOTE - Training set distribution:
calculated_classification
0    379
1    379
2    379
3    379
Name: count, dtype: int64


In [37]:
# Combine features and target into one DataFrame

In [38]:
balanced_data = pd.concat([pd.DataFrame(X_resampled, columns=X_train.columns), y_resampled.reset_index(drop=True)], axis=1)
balanced_data.shape

(1516, 24)

In [39]:
balanced_data.columns

Index(['ageyrs', 'sex', 'finalhiv', 'artyrs', 'fruit', 'veg', 'salt',
       'beverage', 'snacks', 'friedfood', 'creatinine', 'triglycerides',
       'low-density lipoprotein', 'total cholesterol',
       'high-density lipoprotein', 'eGFR_2021', 'vigorous physical activity',
       'moderate physical activity', 'walking', 'smoking', 'bmi',
       'waist_hip_ratio', 'alcohol_consumption', 'calculated_classification'],
      dtype='object')

In [40]:
balanced_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1516 entries, 0 to 1515
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype   
---  ------                      --------------  -----   
 0   ageyrs                      1516 non-null   int64   
 1   sex                         1516 non-null   category
 2   finalhiv                    1516 non-null   category
 3   artyrs                      1516 non-null   float64 
 4   fruit                       1516 non-null   category
 5   veg                         1516 non-null   category
 6   salt                        1516 non-null   category
 7   beverage                    1516 non-null   category
 8   snacks                      1516 non-null   category
 9   friedfood                   1516 non-null   category
 10  creatinine                  1516 non-null   float64 
 11  triglycerides               1516 non-null   float64 
 12  low-density lipoprotein     1516 non-null   float64 
 13  total cholesterol 

In [41]:
# Run CV
summary_df, fold_scores_df = cross_validate_metrics(models, X_resampled, y_resampled)

# View formatted table
print(summary_df.head(14))


Evaluating model: Logistic regression


/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/m

accuracy : 0.5422


/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/m

precision : 0.5241


...

mcc : 0.6082


/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization has

specificity : 0.9033


/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/kassywang/.pyenv/versions/3.10.13/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization has

roc_auc : 0.8919

Evaluating model: KNeighborsClassifier
accuracy : 0.7117
precision : 0.7046
recall : 0.7118
f1 : 0.6925
mcc : 0.6237
specificity : 0.9097
roc_auc : 0.8941

Evaluating model: Gaussian NB
accuracy : 0.4736
precision : 0.5118
recall : 0.4736
f1 : 0.4093
mcc : 0.3313
specificity : 0.8429
roc_auc : 0.7610
                    model              accuracy             precision  \
0     Logistic regression  0.542 (0.529, 0.555)  0.524 (0.507, 0.542)   
1   Suport Vector Machine  0.651 (0.630, 0.671)  0.640 (0.617, 0.661)   
2          Decision trees  0.674 (0.658, 0.691)  0.666 (0.654, 0.683)   
3           Random Forest  0.775 (0.756, 0.788)  0.773 (0.749, 0.788)   
4               Ada Boost  0.537 (0.519, 0.555)  0.543 (0.518, 0.567)   
5       Gradient Boosting  0.752 (0.737, 0.766)  0.745 (0.731, 0.761)   
6   ExtraTrees Classifier  0.744 (0.714, 0.769)  0.735 (0.703, 0.763)   
7                LightGBM  0.787 (0.768, 0.808)  0.789 (0.770, 0.808)   
8                CatBoo

In [42]:
summary_df.to_csv('/Users/kassywang/Documents/Summer Project/R/without outliers/cross_validation_results_after_resampling.csv', index=False)

In [43]:
fold_scores_df['fold'] = fold_scores_df.groupby(['model', 'metric']).cumcount()

pivot_df = fold_scores_df.pivot_table(
    index=['model', 'fold'],
    columns='metric',
    values='score'
).reset_index()

print(pivot_df.head(n =5))

metric      model  fold  accuracy        f1       mcc  precision    recall  \
0       Ada Boost     0  0.565789  0.567112  0.422586   0.574863  0.565789   
1       Ada Boost     1  0.534653  0.527380  0.380608   0.523442  0.535570   
2       Ada Boost     2  0.504950  0.497106  0.344634   0.505904  0.505789   
3       Ada Boost     3  0.551155  0.553136  0.407173   0.576848  0.551140   
4       Ada Boost     4  0.528053  0.527222  0.372272   0.532537  0.527325   

metric   roc_auc  specificity  
0       0.774541     0.855347  
1       0.738502     0.846847  
2       0.753823     0.838311  
3       0.781282     0.850565  
4       0.755292     0.842702  


In [44]:
# Assuming pivot_df is already created as shown before
selected_metrics = ['model', 'mcc', 'roc_auc']
subset_df = pivot_df[selected_metrics]
print(subset_df.head())

metric      model       mcc   roc_auc
0       Ada Boost  0.422586  0.774541
1       Ada Boost  0.380608  0.738502
2       Ada Boost  0.344634  0.753823
3       Ada Boost  0.407173  0.781282
4       Ada Boost  0.372272  0.755292


In [45]:
melted_df = pd.melt(
    subset_df,
    id_vars=['model'],      # columns to keep as identifiers
    value_vars=['mcc', 'roc_auc'],  # columns to unpivot
    var_name='metric',
    value_name='score'
)

melted_df.head(2)

,model,metric,score
0,Ada Boost,mcc,0.422586
1,Ada Boost,mcc,0.380608
